# 类的设计

## OOP和继承： `is-a`关系

In [17]:
class Employee:
    def __init__(self, name, salary=0):
        self.name = name 
        self.salary = salary
    
    def giveRaise(self, percent):
        self.salary = self.salary + (self.salary * percent)
    
    def work(self):
        print(self.name, "does stuff")
    
    def __repr__(self):
        return "<Employee: name=%s, salary=%s>" % (self.name, self.salary)

In [18]:
class Chef(Employee):
    def __init__(self, name):
        Employee.__init__(self, name, 50000)
    
    def work(self):
        print(self.name, "interfaces with customer")

In [28]:
class Server(Employee):
    def __init__(self, name):
        Employee.__init__(self, name)

    def work(self):
        print(self.name, "interfaces with customer")

In [23]:
class PizzaRobot(Chef):
    def __init__(self, name):
        Chef.__init__(self, name)
    
    def work(self):
        print(self.name, "make pizza")

In [24]:
bob = PizzaRobot('bob') # bob是个机器人
print(bob)

<Employee: name=bob, salary=50000>


In [25]:
bob.work()
bob.giveRaise(0.20) # 继承于Employee
print(bob) 
print()

bob make pizza
<Employee: name=bob, salary=60000.0>



In [29]:
for klass in Employee, Chef, Server, PizzaRobot:
    obj = klass(klass.__name__)
    obj.work() # 多态跟对象有关

Employee does stuff
Chef interfaces with customer
Server interfaces with customer
PizzaRobot make pizza


## OOP和组合： `has-a`关系

组合设计把其他对象嵌入容器对象内，并促使其实现容器的方法。组成不是集合的成员关系，而是组件，也就是整体的组成部分。

In [34]:
class Customer:
    def __init__(self, name):
        self.name = name
    
    def order(self, server):
        print(self.name, "order from", server)
    
    def pay(self, server):
        print(self.name, "pays for item to", server)


In [35]:
class Oven:
    def bake(self):
        print("oven bakes")

In [36]:
class PizzaShop:
    def __init__(self):
        self.server = Server('pat')
        self.chef = PizzaRobot('Bob')
        self.oven = Oven()

    def order(self, name):
        customer = Customer(name)
        customer.order(self.server)
        self.chef.work()
        self.oven.bake()
        customer.pay(self.server)

In [37]:
scene = PizzaShop()
scene.order('Homer')
print('...')
scene.order('Shaggy')

Homer order from <Employee: name=pat, salary=0>
Bob make pizza
oven bakes
Homer pays for item to <Employee: name=pat, salary=0>
...
Shaggy order from <Employee: name=pat, salary=0>
Bob make pizza
oven bakes
Shaggy pays for item to <Employee: name=pat, salary=0>


PizzaShop类是容器和控制器；它的构造函数创建并嵌入上一节所编写的员工类，以及这里的Oven类的实例。

每个订单都创建新的Customer对象，并且把内嵌的Server对象传递给Customer的方法，虽然顾客是流动的，但是服务员是比萨店的组件。

## OOP和委托： “包装器”代理对象

委托通常是指控制器对象内嵌其他对象，并把操作请求传递给那些内嵌的对象。控制器能够负责管理类的活动。

通常使用`__getattr__`方法狗子来实现委托，因为这个运算符会拦截对不存在属性的访问，所以包装类可以使用该方法把任意的访问转发给被包装的对象。

In [38]:
class Wrapper:
    def __init__(self, object):
        self.wrapped = object 
    
    def __getattr__(self, attrname):
        print ('Trace: ' + attrname)
        return getattr(self.wrapped, attrname) # 将操作委托给self.warpped本身的属性

In [39]:
x = Wrapper([1, 2, 3])

In [42]:
x.append(4)

Trace: append


In [41]:
x.wrapped

[1, 2, 3, 4]

上例中Wrapper本身没有append方法，通过getattr将append委托给了self.wrapped，因为它是个列表

## 类的伪私有属性

名称重整：在class语句内部，任意开头有双下划线而结尾没有的，会自动在前面包含外围类的名称从而进行扩展。如 `Spam`类的`__x`，会自动变成了`Spam__x`，这样就避免了子类中的变量名冲突。

如果当子类有多个父类时，父类有同名的属性，这样就会冲突，冲突的属性会成为最后赋值的那个。使用伪私有可以解决这个问题

In [43]:
class C1:
    def meth1(self):
        self.__x = 88
    def meth2(self):
        print(self.__x)

In [44]:
class C2:
    def metha(self):
        self.__x = 99
    def methb(self):
        print(self.__x)

In [45]:
class C3(C1, C2):
    pass 

In [48]:
I = C3()

In [53]:
I.meth1()
I.meth2()
I.metha()
I.methb()

88
99


In [55]:
I.__dict__

{'_C1__x': 88, '_C2__x': 99}

## 方法是对象：绑定或未绑定

- 未绑定方法对象：无self， 类进行点号运算获取类的函数属性，回传会未绑定的方法对象。需要显式的传入实例对象
- 绑定方法对象： self+函数， 通过对实例进行点号运算从而获取类的函数属性，会传回绑定方法对象。

In [56]:
class Spam:
    def doit(self, message):
        print(message)

In [58]:
object1 = Spam()
x = object1.doit
x('hello, world') # 这是一个绑定方法对象，通过实例引用的函数，不需要显式的传入实例对象

hello, world


In [59]:
y = Spam.doit
y(object1, 'hello world') # 未绑定的方法对象，通过类直接引用的函数，需要显式传入实例对象才行

hello world


AttributeError: 'Spam' object has no attribute 'y'

绑定方法可以像简单函数一样作为一般对象处理，它们可以在一个程序的各处传递。

In [62]:
class Number:
    def __init__(self, base):
        self.base = base
    
    def double(self):
        return self.base * 2
    
    def triple(self):
        return self.base * 3

In [63]:
x = Number(2)
y = Number(3)
z = Number(4)

In [64]:
x.double()

4

In [65]:
acts = [x.double, y.double, y.triple, z.double]
for act in acts:
    print(act())

4
6
9
8


In [67]:
bound = x.double # 绑定方法对象可以访问其配对实例对象的方法和属性
bound.__self__.base 

2

## 类是对象：一般对象的工厂

有时基于类的设计需要创建对象，以应对程序编写时不能够预测的情况。工厂设计模式允许这样的延迟设计。

In [68]:
def factory(aClass, *pargs, **kargs):
    return aClass(*pargs, **kargs)

In [69]:
class Spam:
    def doit(self, message):
        print(message)

In [70]:
class Person:
    def __init__(self, name, job=None):
        self.name = name 
        self.job = job 

In [72]:
object1 = factory(Spam)
object1.doit(99)

99


In [73]:
object2 = factory(Person, "szq", "king")
object2.name 

'szq'

上例中定义了一个`factory`的对象生成器函数，需要向它传递一个类对象，以及供此类的构造函数使用的一个或多个参数。返回一个类对象的实例

## 多继承：`mix-in`类

多继承擅长为属于多个集合的对象建模，例如一个人既可以是学生，又可以是程序员。。。当时这面临一个问题，就是相同的方法名称在不止一个父类中定义时，会造成冲突。当冲突发生时，冲突要么通过继承搜索顺序自动解决，要么在代码中手动解决。
- 默认情况下，当一个属性被正常引用时，继承选择首次找到的属性。经典类时最低和最左边的。新式类在向上搜索前选择最右边的
- 显式的，通过类名引用它来实现

In [79]:
class ListInstance:
    def __attrnames(self):
        result = ''
        for attr in sorted(self.__dict__):
            result += "\t%s=%s\n" % (attr, self.__dict__[attr])
        return result 
    
    def __str__(self):
        return '<Instance of %s, address %s:\n%s>' %(self.__class__.__name__, id(self), self.__attrnames())

In [80]:
class Spam(ListInstance):
    def __init__(self):
        self.data1 = 'food'

In [81]:
x = Spam()
print(x)

<Instance of Spam, address 2037437219016:
	data1=food
>


由于ListInstance定义了一个`__str__`方法运算符重载方法，因此派生自这个类的实例在打印的时候自动显示自身属性，以及比简单地址多一些信息。对于任何类都很有用，即使类已经拥有一个或多个父类

In [82]:
class Super:
    def __init__(self):
        self.data1 = 'spam'
    def ham(self):
        pass 

In [83]:
class Sub(Super, ListInstance):
    def __init__(self):
        Super.__init__(self)
        self.data2 = 'eggs'
        self.data3 = 42
    
    def spam(self):
        pass 

In [85]:
X =Sub()
print(X)

<Instance of Sub, address 2037438792136:
	data1=spam
	data2=eggs
	data3=42
>


In [89]:
class ListInherited:
    def __attrnames(self):
        result = ''
        for attr in dir(self):
            if attr[:2] == '__' and attr[-2:] == '__':
                result += '\t%s\n' % attr 
            else:
                result += '\t%s=%s\n' %(attr, getattr(self, attr))
        return result 
    
    def __str__(self):
        return '<Instance of %s, address %s:\n%s>' %(self.__class__.__name__, id(self), self.__attrnames())

前面定义的类只能显示实例属性，而不能显示继承的属性。修改后，使用dir和gettatr可以获取继承的属性

In [90]:
class Sub(Super, ListInherited):
    def __init__(self):
        Super.__init__(self)
        self.data2 = 'eggs'
        self.data3 = 42
    
    def spam(self):
        pass 

In [91]:
X =Sub()
print(X)

<Instance of Sub, address 2037445694472:
	_ListInherited__attrnames=<bound method ListInherited.__attrnames of <__main__.Sub object at 0x000001DA613A1408>>
	__class__
	__delattr__
	__dict__
	__dir__
	__doc__
	__eq__
	__format__
	__ge__
	__getattribute__
	__gt__
	__hash__
	__init__
	__init_subclass__
	__le__
	__lt__
	__module__
	__ne__
	__new__
	__reduce__
	__reduce_ex__
	__repr__
	__setattr__
	__sizeof__
	__str__
	__subclasshook__
	__weakref__
	data1=spam
	data2=eggs
	data3=42
	ham=<bound method Super.ham of <__main__.Sub object at 0x000001DA613A1408>>
	spam=<bound method Sub.spam of <__main__.Sub object at 0x000001DA613A1408>>
>
